In [10]:
%pip install IMDbPY

Note: you may need to restart the kernel to use updated packages.


In [1]:
from imdb import IMDb
"""obj = IMDb()
movies = obj.search_movie("The Shawshank Redemption")"""

'obj = IMDb()\nmovies = obj.search_movie("The Shawshank Redemption")'

In [22]:
movie = obj.get_movie(movies[0].movieID)
print(movie['director'])
print(movie['cast'])

[<Person id:0001104[http] name:_Frank Darabont_>]
[<Person id:0000209[http] name:_Tim Robbins_>, <Person id:0000151[http] name:_Morgan Freeman_>, <Person id:0348409[http] name:_Bob Gunton_>, <Person id:0006669[http] name:_William Sadler_>, <Person id:0000317[http] name:_Clancy Brown_>, <Person id:0004743[http] name:_Gil Bellows_>, <Person id:0001679[http] name:_Mark Rolston_>, <Person id:0926235[http] name:_James Whitmore_>, <Person id:0218810[http] name:_Jeffrey DeMunn_>, <Person id:0104594[http] name:_Larry Brandenburg_>, <Person id:0321358[http] name:_Neil Giuntoli_>, <Person id:0508742[http] name:_Brian Libby_>, <Person id:0698998[http] name:_David Proval_>, <Person id:0706554[http] name:_Joseph Ragno_>, <Person id:0161980[http] name:_Jude Ciccolella_>, <Person id:0005204[http] name:_Paul McCrane_>, <Person id:0086169[http] name:_Renee Blaine_>, <Person id:0542957[http] name:_Scott Mann_>, <Person id:0395612[http] name:_John Horton_>, <Person id:2939075[http] name:_Gordon Greene_>,

In [3]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import re 

In [4]:
df = pd.read_csv("data.csv")

In [5]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


### Director and cast have many values missing, country doesnt matter and date added have 100 values missing. Director and cast can be filled using IMDbPY module.

In [37]:
titles = obj.search_movie("The Avengers")

In [46]:
for index, row in df.iterrows():
    print(row.title)
    break

3%


In [12]:
def search_fill(df_filled, title, take_title_match, find_cast, find_dir, cast_df, dir_df):
    titles = imdb_obj.search_movie(title)
    filled = False
    for x in titles:
        if x['title'].lower() != title.lower():
            continue
        movie_data = imdb_obj.get_movie(x.movieID)
        title = movie_data['title']

        try:
            dirs = [d['name'].lower() for j, d in enumerate(movie_data['director']) if j < 2]
        except KeyError:
            dirs = ['Not Found']
            pass
        dirs = ", ".join(dirs)

        try:
            cast = [c['name'].lower() for i, c in enumerate(movie_data['cast']) if i < 5]
        except KeyError:
            cast = ['Not Found']
            pass
        cast = ", ".join(cast)

        if cast ==  ['Not Found'] and dirs == ['Not Found']:
            continue
        i, match_cast, match_dir = 0, 0, 0
        while i < 3:
            if cast_df[i] in cast:
                match_cast +=1
            if dir_df[i] in dirs:
                match_dir +=1
            i +=1

        if match_cast > 0 or match_dir > 0:
            filled = True
            if find_dir:
                df_filled.loc[index, 'director'] = dirs
                print(f"{title} : [dir] {dirs}")


            if find_cast:
                df_filled.loc[index, 'cast'] = cast
                print(f"{title} :  [cast] {cast}")


        elif take_title_match:
            filled = True
            df_filled.loc[index, 'director'] = ", ".join(dirs)
            print(f"{title} : [dir] {dirs}")
            df_filled.loc[index, 'cast'] = ", ".join(cast)
            print(f"{title} :  [cast] {cast}")

        else:
            pass
        

        if filled: 
            del dir_df
            del cast_df
            del take_title_match
            break

In [82]:
df_filled = df.copy()

In [83]:
imdb_obj = IMDb()
from imdb import IMDbDataAccessError
import time

for index, row in df_filled.iterrows():

    cast_type = type(row.cast)
    dir_type = type(row.director)

    
    if cast_type == str and dir_type == str:
        continue 

    title = row.title.lower()
    cont_type = row['type'].lower()
    rel_year = str(row.release_year)

    take_title_match = False
    find_cast = False
    find_dir = False
    cast_df = ['None']*3
    dir_df = ['None']*3
    
    if cast_type != str:
        if dir_type != str:
            take_title_match = True

        elif dir_type == str:
            find_cast = True
            dir_df = [di.lower() for di in list(row.director)]
    else:
        find_dir = True
        cast_df = [r.lower() for r in list(row.cast)]

    retries = 3
    for trial in range(retries):
        try:
            search_fill(df_filled, title, take_title_match, find_cast, find_dir, cast_df, dir_df)
            break

        except IMDbDataAccessError as E:
            print("--"*100)
            print(f"Encountered Data access exception")
            sleep_time = np.random.randint(5, 15)
            print(f"Sleeping for {sleep_time} seconds and retrying...")
            time.sleep(sleep_time)
            print(f"Trial number : {trial}")
            print("--"*100)


3% : [dir] Not Found


2024-03-07 20:57:35,321 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt7371666/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 523, in open
    response = meth(req, response)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 632, in http_response
    response = self.parent.error(
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 555, in e

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 10 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1983 : [dir] Not Found
1994 :  [cast] stefano accorsi, miriam leone, guido caprino, paolo mazzarelli, antonio gerardi
'89 : [dir] mark champine
(T)ERROR :  [cast] khalifah ali al-akili, mike healey, ali kareem, saeed 'shariff' torres
(Un)Well : [dir] Not Found
(Un)Well :  [cast] jennifer julian
#BlackAF : [dir] Not Found
#cats_the_mewvie :  [cast] kevin allocca, phaedra barratt, amanda brennan, mike bridavsky, pamela chen
#Rucker50 :  [cast] inez dickens
100 Days My Prince : [dir] Not Found
100 Things 

2024-03-07 21:06:59,820 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt1291547/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 523, in open
    response = meth(req, response)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 632, in http_response
    response = self.parent.error(
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 555, in e

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 10 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
After Porn Ends :  [cast] amber lynn, asia carrera, bobby slayton, crissy moran, david weintraub
After Porn Ends 3 :  [cast] christy canyon, jenteal, tera patrick, priya rai, bonnie rotten
After the Raid :  [cast] Not Found
Against the Tide : [dir] sarvnik kaur
Age Gap Love : [dir] Not Found
Age of Tanks : [dir] Not Found
Age of Tanks :  [cast] david willey, rachel williams, makhmut gareev, sergey mayev, eric schwartz
Agent : [dir] surender reddy
Agent Raghav : [dir] Not Found
Aggretsuko : [dir] Not Fo

UnboundLocalError: local variable 'cast_df' referenced before assignment

### Code has been imporoved lets start from last title

In [91]:
df[df.title == 'Alien Worlds'].index[0]

398

In [93]:
df_next = df.iloc[398:]

In [94]:
imdb_obj = IMDb()
from imdb import IMDbDataAccessError
import time

for index, row in df_next.iterrows():

    cast_type = type(row.cast)
    dir_type = type(row.director)

    
    if cast_type == str and dir_type == str:
        continue 

    title = row.title.lower()
    cont_type = row['type'].lower()
    rel_year = str(row.release_year)

    take_title_match = False
    find_cast = False
    find_dir = False
    cast_df = ['None']*3
    dir_df = ['None']*3
    
    if cast_type != str:
        if dir_type != str:
            take_title_match = True

        elif dir_type == str:
            find_cast = True
            dir_df = [di.lower() for di in list(row.director)]
    else:
        find_dir = True
        cast_df = [r.lower() for r in list(row.cast)]

    retries = 3
    for trial in range(retries):
        try:
            search_fill(df_filled, title, take_title_match, find_cast, find_dir, cast_df, dir_df)
            break

        except IMDbDataAccessError as E:
            print("--"*100)
            print(f"Encountered Data access exception")
            sleep_time = np.random.randint(5, 15)
            print(f"Sleeping for {sleep_time} seconds and retrying...")
            time.sleep(sleep_time)
            print(f"Trial number : {trial}")
            print("--"*100)


Alien Worlds : [dir] Not Found
Alien Worlds :  [cast] sophie okonedo, leigh joel scott, douglas vakoch, natalie batalha, robert hollingworth
Alive and Kicking :  [cast] hilary alexander, evita arce, kimberly clever, sharon davis, emelie decavita
All About the Washingtons : [dir] Not Found
All American : [dir] Not Found
All for Love : [dir] Not Found
All Hail King Julien : [dir] Not Found
All Hail King Julien: Exiled : [dir] Not Found
All in My Family :  [cast] Not Found
Alone : [dir] john hyams
Alone :  [cast] jules willcox, marc menchaca, anthony heald, jonathan rosenthal, katie o'grady
Alphablocks : [dir] Not Found
AlphaGo :  [cast] ioannis antonoglou, lucas baker, nick bostrom, yoo changhyuk, nam chi-hyung
Altered Carbon : [dir] Not Found
Always a Witch : [dir] Not Found
Amanda Knox :  [cast] amanda knox, meredith kercher, raffaele sollecito, giuliano mignini, nick pisa
Amazing Interiors : [dir] Not Found
Amazing Interiors :  [cast] wesley cannon, nicholas laschkewitsch, john teall,

2024-03-07 22:07:36,063 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0367279/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 523, in open
    response = meth(req, response)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 632, in http_response
    response = self.parent.error(
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 555, in e

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 13 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Arrested Development : [dir] Not Found
Arthdal Chronicles : [dir] Not Found
Ash Vs. Evil Dead : [dir] Not Found
Ashes of Love : [dir] Not Found
Ask the Doctor : [dir] Not Found
Ask the StoryBots : [dir] Not Found
At the Dolphin Bay : [dir] Not Found
Atelier : [dir] elsa maria jakobsdóttir
Athlete A :  [cast] maggie nichols, gina nichols, john nichols, steve berta, marisa kwiatkowski
Atomic Puppet : [dir] Not Found
Attack on Titan : [dir] Not Found
Atypical : [dir] Not Found
Audrie & Daisy :  [cast] rob

2024-03-07 22:10:38,551 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=away&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 7 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Away : [dir] Not Found
Away from Home : [dir] semih kaplanoglu
Ayotzinapa, El paso de la Tortuga :  [cast] ángela buitrago, don bernardo campos, caseta, coyuco, crilin
B: The Beginning : [dir] Not Found
Babies : [dir] Not Found
Babies :  [cast] pascal wallisch
Baby : [dir] Not Found
Baby Ballroom : [dir] Not Found
Babylon Berlin : [dir] Not Found
Bachelor Girls :  [cast] Not Found
Back to 1989 : [dir] Not Found
Back with the Ex : [dir] Not Found
Back with the Ex :  [cast] Not Found
Bad Blood : [dir] Not

...

2024-03-07 22:49:47,111 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt5598206/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 523, in open
    response = meth(req, response)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 632, in http_response
    response = self.parent.error(
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 555, in e

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 7 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Command and Control :  [cast] michael bideller, andreas birnbaum, tom brokaw, robin brosch, harold brown
Community : [dir] Not Found
Conan Without Borders : [dir] Not Found
Conan Without Borders :  [cast] conan o'brien, jordan schlansky, steven yeun, sam richardson, sona movsesian
Concrete Football :  [cast] serge aurier, mehdi benatia, ousmane dembélé, adrien gasmi, gradur
Connected : [dir] benny chan
Conspiracy : [dir] frank pierson
Containment : [dir] Not Found
Control Z : [dir] Not Found
Conversatio

2024-03-07 23:17:04,447 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt10521644/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 523, in open
    response = meth(req, response)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 632, in http_response
    response = self.parent.error(
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 555, in 

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 5 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Extreme Engagement : [dir] Not Found
Extremis :  [cast] monica bhargava, jessica zitter
Ezel : [dir] Not Found
F Is for Family : [dir] Not Found
Fabulous Lives of Bollywood Wives : [dir] Not Found
Fabulous Lives of Bollywood Wives :  [cast] neelam kothari, seema khan, maheep kapoor, bhavana pandey, jet li
Fairy Tail : [dir] Not Found
Fall in Love with Me : [dir] Not Found
Fallet : [dir] Not Found
Falsa identidad : [dir] marta foncilla
Familiar Wife : [dir] Not Found
Family Business : [dir] Not Found
Fam

KeyboardInterrupt: 

In [96]:
df_filled.to_csv("savedat1030.csv")

In [7]:
df[df.title == 'First and Last'].index[0]

2180

In [9]:
df_filled = pd.read_csv("savedat1030.csv")

In [10]:
df_next = df.iloc[2180:]

In [13]:
imdb_obj = IMDb()
from imdb import IMDbDataAccessError
import time

for index, row in df_next.iterrows():

    cast_type = type(row.cast)
    dir_type = type(row.director)

    
    if cast_type == str and dir_type == str:
        continue 

    title = row.title.lower()
    cont_type = row['type'].lower()
    rel_year = str(row.release_year)

    take_title_match = False
    find_cast = False
    find_dir = False
    cast_df = ['None']*3
    dir_df = ['None']*3
    
    if cast_type != str:
        if dir_type != str:
            take_title_match = True

        elif dir_type == str:
            find_cast = True
            dir_df = [di.lower() for di in list(row.director)]
    else:
        find_dir = True
        cast_df = [r.lower() for r in list(row.cast)]

    retries = 3
    for trial in range(retries):
        try:
            search_fill(df_filled, title, take_title_match, find_cast, find_dir, cast_df, dir_df)
            break

        except IMDbDataAccessError as E:
            print("--"*100)
            print(f"Encountered Data access exception")
            sleep_time = np.random.randint(5, 15)
            print(f"Sleeping for {sleep_time} seconds and retrying...")
            time.sleep(sleep_time)
            print(f"Trial number : {trial}")
            print("--"*100)


First and Last : [dir] Not Found
First and Last :  [cast] özge özpirinçci, salih bademci, gulcin kultur sahin, serkan ercan, sacide tasaner
First Team: Juventus : [dir] Not Found
First Team: Juventus :  [cast] d.b. sweeney, juventus f.c., gianluigi buffon, sami khedira, gonzalo higuaín
Fishpeople :  [cast] ray collins, lynne cox, eddie donnellan, matahi drollet, dave rastovich
Fit for Fashion : [dir] Not Found
Fittest in Dubai :  [cast] mat fraser, samantha briggs, sara sigmunsdóttir, laura horvath, jamie greene
Flaked : [dir] Not Found
Flavorful Origins : [dir] Not Found
Flinch : [dir] cameron van hoy
Flint Town : [dir] Not Found
Flint Town :  [cast] tim johnson, bridgette balasko, robert frost, dion reed, devon bernritter
Floor Is Lava : [dir] Not Found
Flowering Heart : [dir] Not Found
Flowers : [dir] Not Found
Follow This : [dir] Not Found
Followers : [dir] james rich
Footprints in the Sand : [dir] ramin mohseni
For Grace :  [cast] curtis duffy
For the Win : [dir] Not Found
Forces 

2024-03-08 11:53:33,011 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=frozen+planet%3A+on+thin+ice&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Pyth

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 11 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Fugitiva : [dir] Not Found
Fukrey Boyzzz : [dir] Not Found
Fuller House : [dir] Not Found
Functional Fitness :  [cast] talayna fortunato, annie mist thorisdottir, adam wiley, jimmy hart, ronnie coleman
Gabby's Dollhouse : [dir] Not Found
Galavant : [dir] Not Found
Game Winning Hit : [dir] Not Found
Gangs of Hassepur : [dir] Not Found
Gargantia on the Verdurous Planet : [dir] Not Found
Generation Iron 2 :  [cast] arnold schwarzenegger, martyn ford, kai greene, rich piana, jay cutler
Generation Iron 3 : 

2024-03-08 11:59:07,825 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt1559038/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 523, in open
    response = meth(req, response)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 632, in http_response
    response = self.parent.error(
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 555, in e

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 9 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
GLOW: The Story of the Gorgeous Ladies of Wrestling :  [cast] emily dole, dee booher, larry whistler, tracee meltzer, angelina altishin
Glowing Embers : [dir] mary jimenez, bénédicte liénard
Go! Go! Cory Carson : [dir] Not Found
Go! Live Your Way : [dir] Not Found
God Eater : [dir] Not Found
Godless : [dir] Not Found
Golden Time :  [cast] yui horie, makoto furukawa, mike haimoto, ai kayano, kaito ishikawa
Gomorrah : [dir] Not Found
Good Girls : [dir] Not Found
Good Morning Call : [dir] Not Found
Good Mo

...

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 8 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
What the Health :  [cast] kip andersen, larry baldwin, neal barnard, tia blanco, david carter
What the Love! with Karan Johar : [dir] Not Found
What We Started :  [cast] steve angello, carl cox, chip eberhart, martin garrix, david guetta
When Calls the Heart : [dir] Not Found
When Hari Got Married :  [cast] Not Found


2024-03-08 15:51:39,970 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt8220344/plotsummary', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 523, in open
    response = meth(req, response)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 632, in http_response
    response = self.parent.error(
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 555, in e

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 8 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
When Heroes Fly : [dir] Not Found
When I See You Again - IMDb : [dir] angel martinez
When the Camellia Blooms : [dir] Not Found
When They See Us : [dir] Not Found
When Two Worlds Collide :  [cast] alberto pizango, alan garcía, aruwihtu pizanku, ander ordoñez, nolver arirama
White Gold : [dir] Not Found
White Lines : [dir] Not Found
White Nights : [dir] taylor hackford
White Rabbit Project : [dir] Not Found
Who Killed Little Gregory? : [dir] Not Found
Who Killed Little Gregory? :  [cast] william salyers,

2024-03-08 15:57:47,800 CRITICAL [imdbpy] c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=woodstock&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\Users\Varshil\AppData\Local\Programs\Python\Python39\lib\urllib\req

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Encountered Data access exception
Sleeping for 10 seconds and retrying...
Trial number : 0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Woodstock :  [cast] richie havens, joan baez, the who, sha-na-na, joe cocker
Word Party : [dir] Not Found
Word Party Songs : [dir] Not Found
Word Party Songs :  [cast] Not Found
Workin' Moms : [dir] Not Found
World at Your Feet : [dir] Not Found
World War II in Colour : [dir] Not Found
World's Busiest Cities : [dir] Not Found
World's Most Wanted : [dir] Not Found
World's Most Wanted :  [cast] jennifer julian, david lorino, pierre-richard prosper, tom fuentes, calogero germaná
Wormwood : [dir] Not Found

In [14]:
df_filled.isnull().sum()

Unnamed: 0        0
show_id           0
type              0
title             0
director        431
cast            136
country         507
date_added       10
release_year      0
rating            7
duration          0
listed_in         0
description       0
dtype: int64

### The work is done and the most values that were found are filled

In [19]:
df_filled[df_filled.director == 'Not Found'].shape

(1464, 13)

### most of the director are not found

In [20]:
df_filled[df_filled.cast == 'Not Found'].shape

(45, 13)

### Most of the cast is found

### The all null values from cast and director should be filled, but some values are just missed and even 'Not Found' is also not filled. Lets fill those values

In [22]:
# lets fill remaining rows with Not Found
df_filled.cast.fillna("Not Found", inplace = True)
df_filled.director.fillna("Not Found", inplace = True)

In [23]:
df_filled.isnull().sum()

Unnamed: 0        0
show_id           0
type              0
title             0
director          0
cast              0
country         507
date_added       10
release_year      0
rating            7
duration          0
listed_in         0
description       0
dtype: int64

In [26]:
df_filled.head(1)

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,s1,TV Show,3%,Not Found,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...


In [27]:
df_filled.drop('Unnamed: 0', axis = 1, inplace = True)

In [28]:
df_filled.to_csv("final_netflix_imputed.csv", index = False)

### **CONCLUSION**

In [29]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

### There were 2389 missing director values. out of those 1895 director values were not found, only 494 values were filled.
### Out of 718 missing cast values 45 were not found and 136 were still null. The filled cast values are 537. 